In [1]:
from nlp import load_dataset
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

In [2]:
train_dataset, test_dataset = load_dataset('emo_data.py', split = ['train', 'test'])

Dataset emo_dataset downloaded and prepared to /home/tanmay/.cache/huggingface/datasets/emo_dataset/tuples of labels and text/1.0.0/2634b69f18155ceec4f4eab8bc9e7b1e39b475e24da38c5bc45d28df7b2b2f12. Subsequent calls will reuse this data.


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 4)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding = True, truncation = True)